databricks Orchestration notebook

In [ ]:
from pyspark.sql.utils import AnalysisException

In [ ]:
notebooks = [
    "/notebooks/3_create_dim_rider",
    "/notebooks/4_create_dim_driver",
    "/notebooks/5_create_dim_location",
    "/notebooks/6_create_dim_date",
    "/notebooks/7_create_fact_trip",
    '/notebooks/8_create_gold_trip_summary',
    'abfss://trip-data@<your-storage-account>.dfs.core.windows.net/gold/monthly_city_kpis'
]

In [ ]:
from spark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
def delta_tables_exist(path):
    try:
        spark.read.format('delta').load(path).limit(1)
        return True
    except AnalysisException:
        return False

In [ ]:
# paths for the above function's testing (corresponding)
output_paths = [
    "abfss://trip-data@<your-storage-account>.dfs.core.windows.net/silver/dim_rider",
    "abfss://trip-data@<your-storage-account>.dfs.core.windows.net/silver/dim_driver",
    "abfss://trip-data@<your-storage-account>.dfs.core.windows.net/silver/dim_location",
    "abfss://trip-data@<your-storage-account>.dfs.core.windows.net/silver/dim_date",
    "abfss://trip-data@<your-storage-account>.dfs.core.windows.net/silver/fact_trip"
]

In [ ]:
# loop through and execute each notebook

force_rerun = False

for i in range(len(notebooks)):
    nb_path = notebooks[i]
    out_path = output_paths[i]
    print(f'---Starting: {nb_path}---')

    # skip if outputs exist
    if not force_rerun and delta_tables_exist(out_path):
        print(f'---Skip(already exists): {out_path}---')
        continue
    try:
        result = dbutils.notebook.run(nb_path,timeout_seconds = 500)
        print(f'---Notebook {nb_path} runs success \n {result}')
    except Exception as e
    print(f'---Run failed: {nb_path} \n Errors: {str(e)}')
    raise e